In [1]:
import torchaudio
import torch
from pathlib import Path


In [2]:

def split_wav(file_path, out_dir, chunk_duration=0.5, target_sr=16000):
    """
    Split a WAV file into fixed-length chunks and save them.

    Parameters:
        file_path (str or Path): Path to input WAV file.
        out_dir (str or Path): Directory to save chunks.
        chunk_duration (float): Duration of each chunk in seconds (default 0.5s).
        target_sr (int): Target sampling rate (default 16000 Hz).
    """
    file_path = Path(file_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load audio
    waveform, sr = torchaudio.load(file_path)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if needed
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        waveform = resampler(waveform)
        sr = target_sr

    num_samples_per_chunk = int(chunk_duration * sr)
    total_samples = waveform.shape[1]
    num_chunks = (total_samples + num_samples_per_chunk - 1) // num_samples_per_chunk

    # Split and save
    for i in range(num_chunks):
        start = i * num_samples_per_chunk
        end = min(start + num_samples_per_chunk, total_samples)
        chunk = waveform[:, start:end]

        # Pad last chunk if too short
        if chunk.shape[1] < num_samples_per_chunk:
            padding = num_samples_per_chunk - chunk.shape[1]
            chunk = torch.nn.functional.pad(chunk, (0, padding))

        out_file = out_dir / f"{file_path.stem}_chunk{i+1}.wav"
        torchaudio.save(str(out_file), chunk, sr)

    print(f"✅ Saved {num_chunks} chunks to {out_dir}")


In [5]:
# split_wav("/home/pierre/Downloads/drone-sound-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
split_wav("/home/pierre/Downloads/this-is-what-a-drone-sounds-like-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")

✅ Saved 51 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa
